# Análise de dados com PySpark

Neste notebook, vamos recriar com PySpark as análises feitas anteriormente com SQL no Hive.<br>
Para tanto, iremos:
1. Baixar e extrair os arquivos utilizados.
2. Configurar e iniciar o ambiente do PySpark.
3. Realizar as análises em PySpark, com as queries SQL servindo de referência comparativa.

## Baixando e extraindo os dados utilizados anteriormente no projeto de Infraestrutura Hadoop

In [7]:
!bash ../scripts/download_and_extract.sh

Iniciando o processo de download e extração em dom 06 out 2024 22:24:14 -03
Baixando o arquivo name.basics.tsv.gz...
--2024-10-06 22:24:14--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolvendo datasets.imdbws.com (datasets.imdbws.com)... 2600:9000:239b:a200:3:3082:af00:93a1, 2600:9000:239b:e400:3:3082:af00:93a1, 2600:9000:239b:4e00:3:3082:af00:93a1, ...
Conectando-se a datasets.imdbws.com (datasets.imdbws.com)|2600:9000:239b:a200:3:3082:af00:93a1|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 273487929 (261M) [binary/octet-stream]
Salvando em: ‘../datasets/name.basics.tsv.gz’

name.basics.tsv.gz    0%[                    ]   1,30M   334KB/s    TED 14m 28s^C
